In [1]:
# for data handling
import pandas as pd
import numpy as np

# train-test split
from sklearn.model_selection import train_test_split

# loss functions for today
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# stuff for evaluating classifiers
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt # for displaying a pretty confusion matrix

# dummy models for comparison
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier

# regression models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

# classification models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

Due to an error in the dataset the airport identifiers from the entirety of October are compromised of intergers instead of the airport ID. This is quite troublesome to fix which is why we've decided to remove the rows from October even though this will defininetly decrease the accuracy of our model.

In [2]:
flight_df = pd.read_csv('flights.csv', dtype={'DESTINATION_AIRPORT': str, 'ORIGIN_AIRPORT': str})

According to the FAA a flight is considered delayed it is delayed by 15 minutes or more.

In [5]:
df_fd = flight_df.copy()
df_fd = df_fd[df_fd.ARRIVAL_DELAY > 15]

We'll clear out some irrelevant columns and retain those we need for our prediction analysis.

In [9]:
# We will not consider cancelled or diverted flights as we are only concerned with delays
df_fd = df_fd[df_fd.CANCELLED != 1]
df_fd = df_fd[df_fd.DIVERTED != 1]

# Drop all columns that do not add interpretability to the model
df_fd = df_fd.drop(['YEAR', 'DEPARTURE_DELAY', 'DEPARTURE_TIME', 'TAXI_OUT','WHEELS_OFF', 'ELAPSED_TIME', 'AIR_TIME',
                  'WHEELS_ON','TAXI_IN','ARRIVAL_TIME','DIVERTED','CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY',
                   'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'], axis=1)
    
# Fill NaNs
df_fd = df_fd.fillna(0)

df_fd.head(30)

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY
27,1,1,4,NK,597,N528NK,MSP,FLL,115,207.0,1487,542,25.0
30,1,1,4,NK,168,N629NK,PHX,ORD,125,204.0,1440,549,43.0
50,1,1,4,B6,1030,N239JB,BQN,MCO,307,173.0,1129,500,20.0
52,1,1,4,B6,2134,N307JB,SJU,MCO,400,185.0,1189,605,85.0
55,1,1,4,B6,2276,N646JB,SJU,BDL,438,241.0,1666,739,89.0
70,1,1,4,AA,1057,N3ASAA,DFW,MIA,515,161.0,1121,856,102.0
73,1,1,4,US,425,N174US,PDX,PHX,520,150.0,1009,850,60.0
74,1,1,4,AA,89,N3KVAA,IAH,MIA,520,141.0,964,841,54.0
86,1,1,4,AA,328,N4XKAA,DEN,DFW,530,125.0,641,835,66.0
92,1,1,4,UA,1532,N77066,SFO,DEN,531,146.0,967,857,26.0
